# List of Libraries

In [37]:
import urllib.request 
import pandas as pd
print(pd.__version__)
import xlrd
print(xlrd.__VERSION__)
import datetime
import progressbar
import requests


1.0.3
1.2.0


# variable declaration

In [38]:
homeUrl = "https://link.springer.com"

# Function to get the pdf/epub link from the baseUrl

In [39]:
def getpdfepubUrl(openUrl):
    split = openUrl.split("/")
    isbn = split[-1]
    pdfUrl = homeUrl + "/content/pdf/" + isbn + ".pdf"
    epubUrl = homeUrl + "/download/epub/" + isbn + ".epub"
    return(pdfUrl,epubUrl)
    
    

# Read excel file - Sheet name - downloadpath (this has the folder name for download

In [40]:
read_df_path = pd.read_excel('Springer_ebook_list.xlsx',
              sheet_name='downloadpath')

In [41]:
dest_path = read_df_path['Download_Destination'].iloc[0]

print(dest_path)

/Users/G_office/Downloads/springer/


# Read excel file - Sheet name  - booklist

In [42]:
read_df_book = pd.read_excel('Springer_ebook_list.xlsx',
              sheet_name='booklist')

In [43]:
#Select those books with has download set as Y or y
options=['Y','y']
download_df = read_df_book[read_df_book['Download(Y/N)'].isin(options)]

In [44]:
download_df

,S.No.,Book_Title,Download(Y/N),Author,Edition,OpenURL
55,56,Microeconomics,y,Peter Dorman,2014,http://link.springer.com/openurl?genre=book&is...
99,100,Computer Vision,Y,Richard Szeliski,2011,http://link.springer.com/openurl?genre=book&is...


In [45]:
download_df['Book_Title'] = download_df['Book_Title'].str.replace(' ','_')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
download_df

,S.No.,Book_Title,Download(Y/N),Author,Edition,OpenURL
55,56,Microeconomics,y,Peter Dorman,2014,http://link.springer.com/openurl?genre=book&is...
99,100,Computer_Vision,Y,Richard Szeliski,2011,http://link.springer.com/openurl?genre=book&is...


In [47]:
output_df = download_df
output_df = download_df.reset_index(drop=True)
output_df


,S.No.,Book_Title,Download(Y/N),Author,Edition,OpenURL
0,56,Microeconomics,y,Peter Dorman,2014,http://link.springer.com/openurl?genre=book&is...
1,100,Computer_Vision,Y,Richard Szeliski,2011,http://link.springer.com/openurl?genre=book&is...


In [48]:
from datetime import datetime

fn_output = "output-" + datetime.now().strftime("%m-%d-%Y_%I-%M-%S_%p") + '.csv'
fn_output

'output-04-29-2020_11-07-12_PM.csv'

## Function to show progress of download

In [ ]:
import progressbar
import urllib.request


pbar = None


def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size).start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None


#urllib.request.urlretrieve(model_url, model_file, show_progress)

In [50]:
for index, row in output_df.iterrows():
    #check for redirects
    r = requests.get(row['OpenURL'])
    pdf_url = getpdfepubUrl(r.url)[0]
    fn_pdf = dest_path  +  row['Book_Title'] + '.pdf'
    print("Processing ======", index, "book title:" , row['Book_Title'] ,fn_pdf)
    try:
        urllib.request.urlretrieve(pdf_url, fn_pdf,show_progress)
        output_df.loc[index,'pdf_status'] = fn_pdf
    except Exception as e:
        output_df.loc[index,'pdf_status'] = str(e)
    epub_url = getpdfepubUrl(r.url)[1]
    #fn_pdf = dest_path  +  row['Book_Title'] + '.pdf'
    fn_epub = dest_path  +  row['Book_Title'] + '.epub'
    try:
        urllib.request.urlretrieve(epub_url, fn_epub, show_progress)
        output_df.loc[index,'epub_status'] = fn_epub
    except Exception as e:
        output_df.loc[index,'epub_status'] = str(e)


output_df.to_csv(open(fn_output,'w'),index=False)

Processing ====== 0 book title: Microeconomics /Users/G_office/Downloads/springer/Microeconomics.pdf


100% |########################################################################|
100% |########################################################################|


Processing ====== 1 book title: Computer_Vision /Users/G_office/Downloads/springer/Computer_Vision.pdf


100% |########################################################################|
